In [ ]:
#default_exp topo_solvers

In [ ]:
#export
import torch
import numpy as np
from tqdm import tqdm

from dl4to.topo_solvers import TopoSolver, SIMPIterator
from dl4to.density_representers import FilteringDensityRepresenter

In [ ]:
#hide
from nbdev.showdoc import show_doc

# SIMP

In [ ]:
#export
class SIMP(TopoSolver):
    """
    A topo solver that performs topology optimization with the "Solid Isotropic Material with Penalization" (SIMP) method.
    """
    def __init__(
        self,
        criterion:"dl4to.criteria.Criterion", # The objective function that should be optimized for in the optimization process.
        p:float=3., # The SIMP exponent to discourage non-binary densities. The default value is `p=3`, which is the standard value in the literature.
        n_iterations:int=50, # The number of SIMP iterations that should be performed.
        verbose:bool=True, # Whether to give the user feedback on the current status of the optimization.
        lr:float=3e-2, # The learning rate of the `torch.optim.Adam` optimizer.
        binarizer_steepening_factor:float=1., # The factor at which the binarizer should be steepened in each iteration. E.g.,a value of 1.1 corresponds to a steepening of 10% per iteration.
        density_representer:"dl4to.density_representers.DensityRepresenter"=FilteringDensityRepresenter(), # The density representer that is used for the latent density representation. The density representer also performs the projection, smoothing and filtering.
        return_intermediate_solutions:bool=False # Whether intermediate SIMP solutions should be returned or only the final solution of the optimization process.
    ):
        super().__init__(device="cpu", name="SIMP")
        self.p = p
        self.n_iterations = n_iterations
        self.verbose = verbose
        self.criterion = criterion
        self.return_intermediate_solutions = return_intermediate_solutions
        self.density_representer = density_representer
        self.binarizer_steepening_factor = binarizer_steepening_factor
        self.lr = lr
        self.criterion = criterion


    def _run_iterations(self, simp_iterator):
        solutions = []
        iters = range(self.n_iterations)
        if self.verbose:
            iters = tqdm(iters)

        for i in iters:
            solution = simp_iterator(p=self.p)
            if self.return_intermediate_solutions:
                solutions.append(solution)
        if self.return_intermediate_solutions:
            return solutions
        return solution


    def reset(self):
        self.density_representer.reset_binarizer()


    def _get_new_simp_iterator(self, solution, density_representer):
        simp_iterator = SIMPIterator(
            problem=solution.problem,
            criterion=self.criterion,
            density_representer=self.density_representer,
            lr=self.lr,
            binarizer_steepening_factor=self.binarizer_steepening_factor
        )
        return simp_iterator


    def _get_new_solution(self, solution):
        self.density_representer.problem = solution.problem
        simp_iterator = self._get_new_simp_iterator(solution, self.density_representer)
        solution = self._run_iterations(simp_iterator)
        return solution


    def _get_new_solutions(self, solutions, density_representers):
        solutions, density_representers = self._check_and_preprocess_inputs_for_get_new_solutions(solutions, density_representers)
        simp_solutions = []
        for solution in solutions:
            if self.return_intermediate_solutions:
                simp_solutions.extend([self._get_new_solution(solution)])
            else:
                simp_solutions.append(self._get_new_solution(solution))
        return simp_solutions


    def _check_and_preprocess_inputs_for_get_new_solutions(self, solutions, density_representers):
        if type(density_representers) != list:
            density_representers = [density_representers]

        if len(solutions) == len(density_representers):
            return solutions, density_representers
        raise ValueError("SIMP: len(solutions) != len(density_representers)")

In [ ]:
#hide
from dl4to.pde import FDM
from dl4to.datasets import BasicDataset
from dl4to.criteria import Compliance, VolumeConstraint

In [ ]:
#hide
max_volume_fraction = 0.1
criterion = Compliance() + VolumeConstraint(max_volume_fraction=.12, threshold_fct='relu')

In [ ]:
#hide
def get_problem():
    problem = BasicDataset(resolution=15).ledge(force_per_area=-4e6)
    problem.pde_solver = FDM()
    return problem

In [ ]:
%%time
#hide

def test_that_we_can_instanciate_and_basic_properties_hold():
    problem = get_problem()
    simp = SIMP(
        criterion=criterion, 
    )

    assert simp.name == "SIMP"


test_that_we_can_instanciate_and_basic_properties_hold()

CPU times: user 323 ms, sys: 16.3 ms, total: 339 ms
Wall time: 354 ms


In [ ]:
%%time
#hide

def test_that_we_can_create_plot(n_iterations, verbose=True):
    problem = get_problem()
    density_representer = FilteringDensityRepresenter(filter_size=3, filter_fct="max_pool")

    simp = SIMP(
        criterion=criterion,
        density_representer=density_representer,
        n_iterations=n_iterations,
        lr=1e-1,
        verbose=True,
    )

    solution = simp(problems_or_solutions=problem)

    if verbose:
        solution.plot()


test_that_we_can_create_plot(n_iterations=3, verbose=False)

  0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 11.4 s, sys: 68 ms, total: 11.5 s
Wall time: 1.08 s
